# Synchronous Generator dq 7th order

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy
import numpy as np

## Steady State - DPsim

In [ ]:
# Define simulation parameters
time_step = 0.0005
final_time = 0.03
sim_name = 'DP_SynGenDq7odODE_SteadyState'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
# resistance for 300 MW output
R_load = 1.92

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQODE('DP_SynGen_dq_SteadyState_SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)
gen.connect([n1])

res = dpsimpy.dp.ph3.SeriesResistor('R_load', dpsimpy.LogLevel.info)
res.set_parameters(R_load)
res.connect([dpsimpy.dp.SimNode.gnd, n1])

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)
sim.run()


In [ ]:
# read log file
work_dir = 'logs/DP_SynGenDq7odODE_SteadyState/'
log_name = 'DP_SynGenDq7odODE_SteadyState'
print(work_dir + log_name + '.csv')
ts_dpsim_ode_ststate = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
ts_dpsim_ode_ststate_emt = ts.frequency_shift_list(ts_dpsim_ode_ststate, 60)

In [ ]:
ts_dpsim_ode_ststate_phasors = ts.phasors(ts_dpsim_ode_ststate)
for node, phasor in ts_dpsim_ode_ststate_phasors.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['v1_0_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['v1_1_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['v1_2_shift'])

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_0_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_1_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_2_shift'])

## Steady State - Simulink

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Simulink/SynchronousGenerator/SynGen_dq_7th_SteadyState/SL_SynGenDq7od_SteadyState_v_i.csv'
local_file = 'reference-results/SL_SynGenDq7od_SteadyState_v_i.csv'
urllib.request.urlretrieve(url, local_file) 

ts_sl_ststate = rt.read_timeseries_simulink(local_file)

In [ ]:
pt.plot_timeseries(1, ts_sl_ststate['i1'])
pt.plot_timeseries(1, ts_sl_ststate['i2'])
pt.plot_timeseries(1, ts_sl_ststate['i3'])
plt.xlim(0,0.03)

## Comparison - Steady State

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_0_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_1_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_ststate_emt['i_load_2_shift'])
pt.plot_timeseries(1, ts_sl_ststate['i1'])
pt.plot_timeseries(1, ts_sl_ststate['i2'])
pt.plot_timeseries(1, ts_sl_ststate['i3'])
plt.xlim(0,0.03)

Time shift is required here to align plots.

## Three Phase Fault - Simulink

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Simulink/SynchronousGenerator/SynGen_dq_7th_ThreePhFault/SL_SynGenDq7od_ThreePhFault_v_i.csv'
local_file = 'reference-results/SL_SynGenDq7od_ThreePhFault_v_i.csv'
urllib.request.urlretrieve(url, local_file) 

ts_sl_tpf = rt.read_timeseries_simulink(local_file)

In [ ]:
pt.plot_timeseries(1, ts_sl_tpf['v1'])
pt.plot_timeseries(1, ts_sl_tpf['v2'])
pt.plot_timeseries(1, ts_sl_tpf['v3'])
plt.xlim(0,0.3)

In [ ]:
pt.plot_timeseries(1, ts_sl_tpf['i1'])
pt.plot_timeseries(1, ts_sl_tpf['i2'])
pt.plot_timeseries(1, ts_sl_tpf['i3'])
plt.xlim(0,0.3)

## Three Phase Fault - DPsim

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odODE_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQODE('DP_SynGen', dpsimpy.LogLevel.debug)
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

In [ ]:
# read log file
work_dir = 'logs/DP_SynGenDq7odODE_ThreePhFault/'
log_name = 'DP_SynGenDq7odODE_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_ode_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
ts_dpsim_ode_tpf_emt = ts.frequency_shift_list(ts_dpsim_ode_tpf, 60)

In [ ]:
ts_dpsim_ode_tpf_phasors = ts.phasors(ts_dpsim_ode_tpf)
for node, phasor in ts_dpsim_ode_tpf_phasors.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

In [ ]:
#ts_dpsim_ode_tpf_emt

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['v1_0_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['v1_1_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['v1_2_shift'])

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_0_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_1_shift'])
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_2_shift'])

### ODE and Simulink

In [ ]:
#%matplotlib qt
ts_sl_tpf_i1 = ts('i1', ts_sl_tpf['i1'].time, -ts_sl_tpf['i1'].values)
ts_sl_tpf_i2 = ts('i2', ts_sl_tpf['i2'].time, -ts_sl_tpf['i2'].values)
ts_sl_tpf_i3 = ts('i3', ts_sl_tpf['i3'].time, -ts_sl_tpf['i3'].values)

#plt.figure(figsize=(20,10))
ts_sl_tpf_i1.label = 'ia_sl'
ts_sl_tpf_i2.label = 'ib_sl'
ts_sl_tpf_i3.label = 'ic_sl'
ts_dpsim_ode_tpf_emt['i_gen_0_shift'].label = 'ia_dpsim'
ts_dpsim_ode_tpf_emt['i_gen_1_shift'].label = 'ib_dpsim'
ts_dpsim_ode_tpf_emt['i_gen_2_shift'].label = 'ic_dpsim'
pt.plot_timeseries(1, ts_sl_tpf_i1, plt_color = '#939393ff')
pt.plot_timeseries(1, ts_sl_tpf_i2, plt_color = '#939393ff')
pt.plot_timeseries(1, ts_sl_tpf_i3, plt_color = '#939393ff')
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_0_shift'], plt_linestyle='-.', plt_color = '#d62728ff')
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_1_shift'], plt_linestyle='-.', plt_color = '#0055ffff')
pt.plot_timeseries(1, ts_dpsim_ode_tpf_emt['i_gen_2_shift'], plt_linestyle='-.', plt_color = '#00aa7fff')
plt.xlabel('time (s)')
plt.ylabel('voltage (V)')
plt.xlim(0.05,0.25)
#plt.ylim(-1.6e5,1.6e5)
#plt.savefig('syngen_fault_ode.pdf')

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ode_tpf_phasors['i_gen_0']['phase'])
plt.xlim([0.09,0.21])

plt.figure()
plt.plot(ts_dpsim_ode_tpf['wr_gen'].time, ts_dpsim_ode_tpf['wr_gen'].values * 60)
plt.xlim([0.09,0.21])

In [ ]:
import numpy as np
diff = ts_sl_tpf_i1.values - ts_dpsim_ode_tpf_emt['i_gen_0_shift'].values
assert np.max(diff[:4000]) < 200

## Three Phase Fault - DPsim Trapez

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odTrapez_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQTrapez('DP_SynGen_dq_ThreePhFault_SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_gen', 'i_intf', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

In [ ]:
# read log file
work_dir = 'logs/DP_SynGenDq7odTrapez_ThreePhFault/'
log_name = 'DP_SynGenDq7odTrapez_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_trapez_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
ts_dpsim_trapez_tpf_emt = ts.frequency_shift_list(ts_dpsim_trapez_tpf, 60)

In [ ]:
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_0_shift'])
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_1_shift'])
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_2_shift'])

### Trapez and Simulink

In [ ]:
ts_sl_tpf_i1 = ts('i1', ts_sl_tpf['i1'].time, -ts_sl_tpf['i1'].values)
ts_sl_tpf_i2 = ts('i2', ts_sl_tpf['i2'].time, -ts_sl_tpf['i2'].values)
ts_sl_tpf_i3 = ts('i3', ts_sl_tpf['i3'].time, -ts_sl_tpf['i3'].values)

#plt.figure(figsize=(20,10))
pt.plot_timeseries(1, ts_sl_tpf_i1)
pt.plot_timeseries(1, ts_sl_tpf_i2)
pt.plot_timeseries(1, ts_sl_tpf_i3)
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_0_shift'])
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_1_shift'])
pt.plot_timeseries(1, ts_dpsim_trapez_tpf_emt['i_gen_2_shift'])

In [ ]:
import numpy as np
diff = ts_sl_tpf_i1.values - ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].values
assert np.max(diff[:4000]) < 200